In [1]:
import os, shutil, glob, time
import pandas as pd
import numpy as np


os.environ["HADOOP_HOME"] = "D:/hadoop"
os.environ["PATH"] += os.pathsep + "D:/hadoop/bin"
os.makedirs("D:/hadoop/checkpoint", exist_ok=True)

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, from_unixtime
from pyspark.sql.types import StructType, IntegerType, DoubleType, TimestampType

from sqlalchemy import create_engine
import urllib

In [3]:
# Cấu hình thư mục
output_dir = "D:/sensor-data/output"
checkpoint_dir = "D:/hadoop/checkpoint"
backup_dir = "D:/sensor-data/backup"

# Tạo thư mục nếu chưa có
os.makedirs(output_dir, exist_ok=True)
os.makedirs(checkpoint_dir, exist_ok=True)
os.makedirs(backup_dir, exist_ok=True)


def merge_csv_files_clean_headers(input_dir, output_file):
    csv_files = glob.glob(os.path.join(input_dir, "part-*"))
    if not csv_files:
        print("⚠️ Không có file để gộp.")
        return None

    dfs = []
    for file in csv_files:
        try:
            df = pd.read_csv(file, header=None)
            df = df[df[0] != 'sensor_id']
            dfs.append(df)
        except Exception as e:
            print(f"❌ Lỗi đọc file {file}: {e}")

    if dfs:
        merged_df = pd.concat(dfs, ignore_index=True)
        merged_df.columns = ['sensor_id', 'temperature', 'humidity', 'timestamp']
        merged_df.to_csv(output_file, index=False)
        print(f"✅ Đã gộp và làm sạch thành {output_file}")
        return True
    else:
        print("⚠️ Không có nội dung hợp lệ để gộp.")
        return False


# Xoá output và checkpoint cũ
def cleanup_dirs():
    shutil.rmtree(output_dir, ignore_errors=True)
    shutil.rmtree(checkpoint_dir, ignore_errors=True)
    print("🧹 Đã xoá output và checkpoint.")


In [4]:
import pandas as pd

def validate_data(file_path):
    try:
        df = pd.read_csv(file_path)

        # 1. Kiểm tra không có giá trị null
        if df['sensor_id'].isnull().any():
            print("❌ Có giá trị thiếu trong 'sensor_id'")
            return False
        if df['temperature'].isnull().any():
            print("❌ Có giá trị thiếu trong 'temperature'")
            return False
        if df['humidity'].isnull().any():
            print("❌ Có giá trị thiếu trong 'humidity'")
            return False
        if df['timestamp'].isnull().any():
            print("❌ Có giá trị thiếu trong 'timestamp'")
            return False

        # 2. Kiểm tra range nhiệt độ (-50°C đến 100°C)
        if not df['temperature'].between(-50, 100).all():
            print("❌ Nhiệt độ ngoài khoảng hợp lệ (-50 đến 100°C)")
            return False

        # 3. Kiểm tra độ ẩm (0% đến 100%)
        if not df['humidity'].between(0, 100).all():
            print("❌ Độ ẩm ngoài khoảng hợp lệ (0 đến 100%)")
            return False

        # 4. Kiểm tra định dạng thời gian (yyyy-mm-dd HH:MM:SS)
        try:
            pd.to_datetime(df['timestamp'], format="%Y-%m-%d %H:%M:%S", errors='raise')
        except ValueError:
            print("❌ Định dạng 'timestamp' không hợp lệ")
            return False

        print("✅ Tất cả kiểm tra dữ liệu đều đạt yêu cầu.")
        return True

    except Exception as e:
        print(f"❌ Lỗi khi đọc hoặc kiểm tra dữ liệu: {e}")
        return False


In [5]:
# SparkSession
spark = SparkSession.builder \
    .appName("KafkaSensorConsumerWithValidation") \
    .master("local[*]") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.2") \
    .config("spark.hadoop.home.dir", "D:/hadoop") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

# Schema dữ liệu Kafka
schema = StructType() \
    .add('sensor_id', IntegerType()) \
    .add('temperature', DoubleType()) \
    .add('humidity', DoubleType()) \
    .add('timestamp', DoubleType())  # timestamp dạng Unix

# Đọc dữ liệu từ Kafka
df_raw = spark.readStream \
    .format('kafka') \
    .option('kafka.bootstrap.servers', 'localhost:9092') \
    .option('subscribe', 'sensor-data') \
    .option('startingOffsets', 'latest') \
    .load()

# Parse JSON từ Kafka và chuyển timestamp sang dạng string
df_parsed = df_raw.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema).alias("data")) \
    .select("data.*") \
    .withColumn("timestamp", from_unixtime(col("timestamp")).cast("string"))

# Ghi dữ liệu ra CSV nếu hợp lệ
query = df_parsed.writeStream \
    .option("path", output_dir) \
    .option("checkpointLocation", checkpoint_dir) \
    .option("header", True) \
    .format("csv") \
    .start()

query.awaitTermination(120)  # 2 phút
query.stop()
print("🔴 Đã dừng stream sau 2 phút.")

🔴 Đã dừng stream sau 2 phút.


In [6]:
# ==== GỘP FILE + KIỂM TRA DỮ LIỆU ====
timestamp_str = time.strftime("%Y%m%d_%H%M%S")
merged_file_path = f"{backup_dir}/merged_{timestamp_str}.csv"

# Hàm merge trả về True nếu gộp thành công
success = merge_csv_files_clean_headers(output_dir, merged_file_path)

if success and os.path.exists(merged_file_path):
    is_clean = validate_data(merged_file_path)
    if is_clean:
        print("✅ Dữ liệu đạt yêu cầu chất lượng, có thể đưa vào SQL/Azure.")
    else:
        print("❌ Dữ liệu KHÔNG đạt chất lượng. Hủy đẩy vào hệ thống chính.")
    cleanup_dirs()  # Xoá output và checkpoint sau khi kiểm tra
else:
    print("⚠️ Gộp file thất bại hoặc không tồn tại file đầu ra. Bỏ qua kiểm tra.")


✅ Đã gộp và làm sạch thành D:/sensor-data/backup/merged_20250616_152345.csv
✅ Tất cả kiểm tra dữ liệu đều đạt yêu cầu.
✅ Dữ liệu đạt yêu cầu chất lượng, có thể đưa vào SQL/Azure.
🧹 Đã xoá output và checkpoint.


In [7]:
# File CSV sau khi gộp
csv_path = merged_file_path

# Đọc dữ liệu
df = pd.read_csv(csv_path)

# Tên instance SQL Server trong máy bạn (Express hoặc mặc định)
server = "LAPTOP-CUA-QUAN\SQLSERVER1"  # hoặc chỉ "localhost" nếu dùng mặc định
database = "SensorData"

# Tạo connection string
connection_string = (
    f"mssql+pyodbc://{server}/{database}"
    "?driver=ODBC+Driver+17+for+SQL+Server"
    "&trusted_connection=yes"
)

# Tạo engine và đẩy dữ liệu
engine = create_engine(connection_string)
df.to_sql('sensor_data', con=engine, if_exists='append', index=False)

print("✅ Dữ liệu đã được đẩy vào SQL Server thành công.")


✅ Dữ liệu đã được đẩy vào SQL Server thành công.


In [16]:
# Sau khi validate dữ liệu
if success and os.path.exists(merged_file_path):
    is_clean = validate_data(merged_file_path)
    
    run_time = pd.Timestamp.now()
    record_count = len(df)
    job_name = "KafkaSensorETL"

    if is_clean:
        
        # Ghi log thành công
        log_df = pd.DataFrame({
            'start_time': [run_time],
            'end_time': [run_time],
            'job_name': [job_name],
            'status': ['Success'],
            'records_processed': [record_count],
            'error_message': ['Data validation passed']
        })
        log_df.to_sql('ETL_Log', con=engine, if_exists='append', index=False)
        print("📝 Đã ghi log ETL thành công vào ETL_Log.")

    else:

        # Ghi log thất bại với thông báo lỗi
        log_df = pd.DataFrame({
            'start_time': [run_time],
            'end_time': [run_time],
            'job_name': [job_name],
            'status': ['Failed'],
            'records_processed': [record_count],
            'error_message': ['Data validation failed']
        })
        log_df.to_sql('ETL_Log', con=engine, if_exists='append', index=False)
        print("📝 Đã ghi log ETL thất bại vào ETL_Log.")

else:
    print("⚠️ Gộp file thất bại hoặc không tồn tại file đầu ra. Bỏ qua kiểm tra.")


✅ Tất cả kiểm tra dữ liệu đều đạt yêu cầu.
📝 Đã ghi log ETL thành công vào ETL_Log.
